In [ ]:
import pandas
import os
files={'RUL':['RUL_FD001.txt','RUL_FD002.txt','RUL_FD003.txt','RUL_FD004.txt'],
        'test':['test_FD001.txt','test_FD002.txt','test_FD003.txt','test_FD004.txt'],
        'train':['train_FD001.txt','train_FD002.txt','train_FD003.txt','train_FD004.txt']}
data = pandas.read_csv(os.path.join('data/CMAPSS',files['RUL'][0]))